In [12]:
# Import installed modules
import urllib
from pathlib import Path

import numpy as np
import pandas as pd


def download_file(dl_path, dl_url):
    """ Download files from the Internet.

    Parameters
    ----------
    dl_path : str
        Download path on the local machine, relative to this function.
    dl_url : str
        Download url of the requested file.
    """

    # Make directory if it does not exist
    dl_path = Path(dl_path)
    dl_path.mkdir(parents=True, exist_ok=True)
    # Get the file name from the url
    file_name = dl_url.split('/')[-1]

    # If the file is not present in the download directory -> download it
    if not (dl_path/file_name).exists():
        # Download the file with the urllib  package
        urllib.request.urlretrieve(dl_url, dl_path/file_name)


# Download Rosetta 67P shape model (decent high-resolution, 75MB; took 8min for me.)
download_file(
    "data/",
    'https://naif.jpl.nasa.gov/pub/naif/ROSETTA/kernels/dsk/ROS_CG_M001_OSPCLPS_N_V1.OBJ'
)

shape_67p = pd.read_csv('data/ROS_CG_M001_OSPCLPS_N_V1.OBJ', delim_whitespace=True, names=['typ', 'x', 'y', 'z'])
num_v = np.count_nonzero(shape_67p["typ"]=="v")
num_f = np.count_nonzero(shape_67p["typ"]=="f")
print("vertices and faces: ", num_v, num_f)
print(shape_67p.head())
print(shape_67p.tail())

vertices and faces:  597251 1194496
  typ         x         y         z
0   v  0.570832 -1.000444  0.532569
1   v  0.564360 -1.000224  0.525360
2   v  0.557853 -0.997863  0.520762
3   v  0.553592 -0.998414  0.512192
4   v  0.550212 -0.992514  0.507304
        typ         x         y         z
1791742   f  502024.0  596634.0  502287.0
1791743   f  502287.0  597090.0  502288.0
1791744   f  597090.0  501842.0  502288.0
1791745   f  597090.0  597234.0  501842.0
1791746   f  597234.0  429305.0  501842.0


In [16]:
verts = shape_67p[shape_67p["typ"]=="v"].values[:,1:].astype(float)
faces = shape_67p[shape_67p["typ"]=="f"].values[:,1:].astype(int) - 1  # -1 because OBJ starts counting at 1
faces

array([[   473,    521,    304],
       [   473,    718,    521],
       [  1960,   2159,   1650],
       ...,
       [597089, 501841, 502287],
       [597089, 597233, 501841],
       [597233, 429304, 501841]])

In [5]:
# Assign the VERTICES and faces
VERTICES = COMET_67P_SHAPE_OBJ.loc[COMET_67P_SHAPE_OBJ['TYPE'] == 'v'][['X1', 'X2', 'X3']].values \
               .tolist()
faces = COMET_67P_SHAPE_OBJ.loc[COMET_67P_SHAPE_OBJ['TYPE'] == 'f'][['X1', 'X2', 'X3']].values

In [6]:
# Print the minimum and maximum vertex indices in the face sub set
print(f'Minimum vertex index in faces: {np.min(faces)}')
print(f'Maximum vertex index in faces: {np.max(faces)}')
print('\n')

Minimum vertex index in faces: 1.0
Maximum vertex index in faces: 597251.0




In [7]:
# The index in the faces sub set starts at 1. For Python, it needs to start at 0.
faces = faces - 1

# Convert the indices to integer
faces = faces.astype(int)

# Convert the numpy array to a Python list
faces = faces.tolist()

In [8]:
# Now we need to define a main window class that is needed to set a window size / resolution.
# Based on the QT4 example:
# https://github.com/almarklein/visvis/blob/master/examples/embeddingInQt4.py
from PyQt5.QtWidgets import QWidget, QHBoxLayout

# Import visvis
import visvis as vv

# Define the class
class MainWindow(QWidget):
    def __init__(self, *args):
        QWidget.__init__(self, *args)
        self.fig = vv.backends.backend_pyqt5.Figure(self)
        self.sizer = QHBoxLayout(self)
        self.sizer.addWidget(self.fig._widget)
        self.setLayout(self.sizer)
        self.setWindowTitle('Rosetta')
        self.show()

In [9]:
# Create visvis application
app = vv.use()
app.Create()

# Create main window frame and set a resolution.
main_w = MainWindow()
main_w.resize(1200, 800)

# Create the 3 D shape model as a mesh. verticesPerFace equals 3 since triangles define the
# mesh's surface in this case
vv.mesh(vertices=VERTICES, faces=faces, verticesPerFace=3)

# Get axes objects
axes = vv.gca()

# Set a black background
axes.bgcolor = 'black'

# Deactivate the grid and make the x, y, z axes invisible
axes.axis.showGrid = False
axes.axis.visible = False

# Set some camera settings
# Please note: if you want to "fly" arond the comet with w, a, s, d (translation) and i, j, k, l
# (tilt) replace '3d' with 'fly'
axes.camera = '3d'

# Field of view in degrees
axes.camera.fov = 60

# Set default azmiuth and elevation angle in degrees
axes.camera.azimuth = 120
axes.camera.elevation = 25

# ... and run the application!
app.Run()

In [10]:
# Now let's create an animation

# Create visvis application
app = vv.use()
app.Create()

# Create main window frame and set a resolution.
main_w = MainWindow()
main_w.resize(500, 400)

# Create the 3 D shape model as a mesh. verticesPerFace equals 3 since triangles define the
# mesh's surface in this case
shape_obj = vv.mesh(vertices=VERTICES, faces=faces, verticesPerFace=3)
shape_obj.specular = 0.0
shape_obj.diffuse = 0.9

# Get figure
figure = vv.gcf()

# Get axes objects and set figure parameters
axes = vv.gca()
axes.bgcolor = (0, 0, 0)
axes.axis.showGrid = False
axes.axis.visible = False

# Set camera settings
#
axes.camera = '3d'
axes.camera.fov = 60
axes.camera.zoom = 0.1

# Turn off the main light
axes.light0.Off()

# Create a fixed light source
light_obj = axes.lights[1]
light_obj.On()
light_obj.position = (5.0, 5.0, 5.0, 0.0)

# Empty array that contains all images of the comet's rotation
comet_images = []

# Rotate camera in 300 steps in azimuth
for azm_angle in tqdm(range(300)):

    # Change azimuth angle of the camera
    axes.camera.azimuth = 360 * float(azm_angle) / 300

    # Draw the axes and figure
    axes.Draw()
    figure.DrawNow()

    # Get the current image
    temp_image = vv.getframe(vv.gca())

    # Apped the current image in 8 bit integer
    comet_images.append((temp_image*255).astype(np.uint8))

# Save the images as an animated GIF
imageio.mimsave('Comet67P.gif', comet_images, duration=0.04)

100%|██████████| 300/300 [00:10<00:00, 27.90it/s]
